In [19]:
import numpy as np
import caffe
from caffe_io import transform_image
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

In [20]:
image = caffe.io.load_image(
    '/Users/ecsark/Documents/visualdb/project/wikiart/images/orest-kiprensky_young-gardener-1817.jpg')
    #'/Users/ecsark/Documents/visualdb/project/wikiart/images/salvador-dali_still-life-pulpo-y-scorpa.jpg')
    #'/Users/ecsark/Documents/visualdb/project/wikiart/images/david-burliuk_in-the-park.jpg')

In [30]:
caffe_root = '/Users/ecsark/Documents/visualdb/caffe/'
model_weights = caffe_root + 'models/vgg/model.caffemodel'
model_def = caffe_root + 'models/vgg/deploy.prototxt'
net = caffe.Net(model_def, model_weights, caffe.TEST)
net.blobs['data'].reshape(1, 3, 224, 224)

In [31]:
mu = np.array([104.0, 117.0, 123.0])
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

In [32]:
def reshape_image(image, new_dim=256, crop_dim=224):
    dsize = (new_dim, new_dim)
    crop_shape = (crop_dim, crop_dim)
    reshaped_img = cv2.resize(image, dsize)
    crop_amt = (new_dim-crop_dim) / 2
    return reshaped_img[crop_amt:crop_amt+crop_dim, crop_amt:crop_amt+crop_dim]

In [33]:
net.blobs['data'].data[...] = transformer.preprocess('data', reshape_image(image, new_dim=256, crop_dim=224))
_ = net.forward()
print net.blobs['fc7'].data

[[ 0.49621016  0.          0.         ...,  0.          0.          0.        ]]


In [ ]:
type(fc7)

In [ ]:
import cPickle as pickle
fc7 = pickle.load(open('/Users/ecsark/Documents/visualdb/project/artwork-explorer/experiment/fc7.pk','rb'))

In [ ]:
fc7['features']

In [ ]:
transformed_image = transform_image(image, False, [104,117,123], 256, 224)